<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/navernews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 31.0 MB/s eta 0:00:00


**News Crawler**

In [75]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, page=1, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    pages = list(range(1,page+1))
    li = []
    for page in pages:
      url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")

      div = bs_obj.find_all('div', {'class':'hotNewsList'})
      li+=[html.unescape(x['href']) for x in div[0].find_all('a')]
    return li

  def get_main(self, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/mainnews.nhn'
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return [x.find_all('a')[0]['href'] for x in bs_obj.find_all('dd', {'class':'articleSubject'})]

  # 시황 전망: 401
  # 기업 종목분석: 402
  # 해외 증시: 403
  # 채권 선물: 404
  # 공시 메모: 406
  # 환율: 429
  def get_focus(self, page=1, code=402, date='1', ):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    pages = range(0,page)
    li = []
    for page in pages:
      url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3='\
      + str(code) +  '&date=' + str(date) + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")
      temp = [html.unescape(x.find_all('a')[0]['href']) for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
      # 업데이트 가능
      li+=[x.replace('§', '&sect') for x in temp]
    return li

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    text = text.replace(subheading, '')
    return title, subheading, text

  def get_code(self, code):
    page = 5
    li = []
    for i in range(1, page):
      url = 'https://finance.naver.com/item/news_news.naver?code=' + code + '&page=' + str(i) + '&sm=title_entity_id.basic&clusterId='
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")
      tds = bs_obj.find_all('td', {'class': 'title'})
      texts = [td.text.strip() for td in tds]
      li.extend([sentence.replace('[', ' ').replace(']', ' ').replace('…', ' ').replace('·', ' ').replace('"', ' ').replace('“', ' ').replace(',', ' ').replace('...', ' ') for sentence in texts])
    return li


In [76]:
from multiprocessing import Pool

news = News()
news_list = news.get_ranking(5)
with Pool() as p:
    li = p.map(news.get_article, news_list)
li = np.array(li)

dic = {'Title':li[:,0], 'Subheading':li[:,1], 'Text':li[:,2]}
df = pd.DataFrame.from_dict(dic)
df

,Title,Subheading,Text
0,“8만전자 못 기다려”…연일 연고점 경신에도 개미는 1조 넘게 순매도,"삼성전자, 이달 개인투자자 코스피 순매도 1위7만원대 안착 상승 기조에도 매도 우위...",[이데일리 원다연 기자] 삼성전자(005930) 주가가 7만원대에 안착해 상승 기조...
1,"“이걸 왜 사죠?”…내주 상폐 앞둔 우선주, 또 동반 급등",,사진은 기사 내용과 직접적인 관련이 없음. [출처 : 연합뉴스]상장주식수 부족으로 ...
2,CJ·네카오 죽쑤자…'몰빵' 콘텐츠 ETF들 '날벼락',"'KODEX Fn웹툰&드라마', 이달 10%대 급락'TIGER 미디어컨텐츠'도 8%...",[서울경제] 네이버(NAVER(035420))·카카오(035720)와 CJ그룹주 주...
3,"""美 증시 써머랠리 이미 시작""...7월이 기대되는 이유",,미국 투자자문사 카슨 그룹이 미국 증시 써머랠리가 이미 시작됐다고 평가했다.28일(...
4,"""9월 美기준금리 5.75%일 수도""…주춤한 나스닥 0.27%↑[뉴욕마감]",,(워싱턴 로이터=뉴스1) 우동명 기자 = 제롬 파월 연방준비제도(Fed) 의장이 1...
...,...,...,...
116,"[특징주] 메가스터디, 세무조사 소식에 약 3% 하락",,"국세청, 메가스터디 특별 세무조사 실시(서울=연합뉴스) 김도훈 기자 = 국세청이 사..."
117,"시큐센, 공모가 대비 220% 상승…개장 직후 '따따블'은 실패",오늘 코스닥 신규 상장…상장일 가격변동폭 확대 조치 '첫 타자',시큐센[IPR스퀘어 제공. 재판매 및 DB 금지] (서울=연합뉴스) 송은경 기자 =...
118,"루네이트, 더블 타이틀곡 '보야저' 퍼포먼스 MV 추가 공개",,판타지오는 보이그룹 루네이트가 지난 28일 오후 8시 공식 유튜브 채널에 첫 번째 ...
119,"HLB ""리보세라닙 병용요법, 환자 위험비·치료중단율 모두 역대 최저""",,HLB는 2008년부터 2022년까지 진행된 15건의 간암 1차 치료제 임상 3상 ...


In [77]:
df

,Title,Subheading,Text
0,“8만전자 못 기다려”…연일 연고점 경신에도 개미는 1조 넘게 순매도,"삼성전자, 이달 개인투자자 코스피 순매도 1위7만원대 안착 상승 기조에도 매도 우위...",[이데일리 원다연 기자] 삼성전자(005930) 주가가 7만원대에 안착해 상승 기조...
1,"“이걸 왜 사죠?”…내주 상폐 앞둔 우선주, 또 동반 급등",,사진은 기사 내용과 직접적인 관련이 없음. [출처 : 연합뉴스]상장주식수 부족으로 ...
2,CJ·네카오 죽쑤자…'몰빵' 콘텐츠 ETF들 '날벼락',"'KODEX Fn웹툰&드라마', 이달 10%대 급락'TIGER 미디어컨텐츠'도 8%...",[서울경제] 네이버(NAVER(035420))·카카오(035720)와 CJ그룹주 주...
3,"""美 증시 써머랠리 이미 시작""...7월이 기대되는 이유",,미국 투자자문사 카슨 그룹이 미국 증시 써머랠리가 이미 시작됐다고 평가했다.28일(...
4,"""9월 美기준금리 5.75%일 수도""…주춤한 나스닥 0.27%↑[뉴욕마감]",,(워싱턴 로이터=뉴스1) 우동명 기자 = 제롬 파월 연방준비제도(Fed) 의장이 1...
...,...,...,...
116,"[특징주] 메가스터디, 세무조사 소식에 약 3% 하락",,"국세청, 메가스터디 특별 세무조사 실시(서울=연합뉴스) 김도훈 기자 = 국세청이 사..."
117,"시큐센, 공모가 대비 220% 상승…개장 직후 '따따블'은 실패",오늘 코스닥 신규 상장…상장일 가격변동폭 확대 조치 '첫 타자',시큐센[IPR스퀘어 제공. 재판매 및 DB 금지] (서울=연합뉴스) 송은경 기자 =...
118,"루네이트, 더블 타이틀곡 '보야저' 퍼포먼스 MV 추가 공개",,판타지오는 보이그룹 루네이트가 지난 28일 오후 8시 공식 유튜브 채널에 첫 번째 ...
119,"HLB ""리보세라닙 병용요법, 환자 위험비·치료중단율 모두 역대 최저""",,HLB는 2008년부터 2022년까지 진행된 15건의 간암 1차 치료제 임상 3상 ...


In [78]:
# korean font
!wget 'https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKkr-hinted.zip'
!unzip "NotoSansCJKkr-hinted.zip"
!mv NotoSansCJKkr-Medium.otf /usr/share/fonts/truetype/

--2023-06-29 06:51:53--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKkr-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c04::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121163981 (116M) [application/zip]
Saving to: ‘NotoSansCJKkr-hinted.zip’

NotoSansCJKkr-hinte 100%[===================>] 115.55M  30.8MB/s    in 4.6s    

2023-06-29 06:51:58 (25.4 MB/s) - ‘NotoSansCJKkr-hinted.zip’ saved [121163981/121163981]

Archive:  NotoSansCJKkr-hinted.zip
  inflating: LICENSE_OFL.txt         
  inflating: NotoSansCJKkr-Black.otf  
  inflating: NotoSansCJKkr-Bold.otf  
  inflating: NotoSansCJKkr-DemiLight.otf  
  inflating: NotoSansCJKkr-Light.otf  
  inflating: NotoSansCJKkr-Medium.otf  
  inflating: NotoSansCJKkr-Regular.otf  
  inflating: NotoSansCJKkr-Thi

In [79]:
# https://wikidocs.net/22530
from konlpy.tag import Hannanum
from wordcloud import WordCloud
import matplotlib.pyplot as plt

class Analyser:
  def __init__(self, df):
    self.df = df
    self.words = ''
    self.stop_words = ['있다', '기자', '다만', '등의', '것으로', '기대감', '기대감에', '특징주', '실적', '기대', '최대', '신영', '순매수', '상위종목', '마감', \
                  '코스닥', '규모', '기관', '외국인', '신영증권', '이데일리', '연합뉴스', '서울경제', '하이투자증권', 'NH투자증권', '뉴스1', '영상', '서울', \
                  '뉴시스', '고', '며', '포인트', '라', '사진', '현지시간', 'S', 'SP', '헤럴드경제', '사진제공', '것', '이라', '로부터', '때문', '제공', 'P', '애널리스트']
    self.wordcloud = ''

  def get_tokenized(self, text):
    hannanum = Hannanum()
    return hannanum.morphs(text)

  def get_cleaned(self, text):
    text = ([x for x in text if len(x) > 1])
    d = {i:text.count(i) for i in set(text) if text.count(i) > 1}
    words = ' '.join(d.keys())
    return words

  def get_wordcloud(self, column):
    t = analyser.df[column]
    words = self.get_tokenized(' '.join(t))
    words = self.get_cleaned(words)
    words = words.split()
    words = ' '.join(words)
    self.words = words
    wordcloud = WordCloud(stopwords = self.stop_words, \
                          font_path='/usr/share/fonts/truetype/NotoSansCJKkr-Medium.otf', \
                          background_color='white', \
                          width=1200, height=800).generate(words)
    self.wordcloud = wordcloud
    plt.figure(figsize=(18,10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")

In [87]:
news = News()
s = news.get_code('001720')

In [88]:
import nltk

In [90]:
# Create a list of words from the sentences
words = [word for sentence in s for word in sentence.split()]

# Add the stop words
stop_words = ['%', '에', '전망', '수혜', '추가', '주가', '주식', '특징주', \
              '코스닥', '코스피', '시총', '전년비', '1분기', '2분기', '3분기', \
              '4분기', '전년比', '↑', '지난해', '영업익']

# Remove stop words from the list of words
words = [word for word in words if word not in stop_words]

# Count the frequency of each word in the list
word_frequencies = nltk.FreqDist(words)

# Get the 5 most frequent words
keywords = word_frequencies.most_common(7)

# Print the 5 keywords
[x[0] for x in keywords[1:]]

['인사', '체결', '유산기부신탁', '퇴직연금', '디폴트옵션', '업무협약']